# Proyecto OilyGiant

<a id='introduccion'></a>
## Introducción

La compañía de extracción de petróleo OilyGiant se encuentra en la búsqueda de los mejores lugares para la apertura de 200 nuevos pozos de petróleo. Para lograr este objetivo, en este estudio se procederá a la lectura de los archivos que contienen los parámetros recopilados de los pozos petrolíferos en la región seleccionada, los cuales incluyen información sobre la calidad del crudo y el volumen de reservas. Posteriormente, se desarrollará un modelo predictivo para estimar el volumen de reservas en los nuevos pozos, con el objetivo de seleccionar aquellos que presenten los valores estimados más favorables. Finalmente, se tomará la decisión estratégica de elegir la región que ofrezca el beneficio total más alto para los pozos petrolíferos seleccionados, en aras de maximizar la rentabilidad y eficiencia en las operaciones de extracción de petróleo.

[Volver al inicio](#contenido)

<a id='preparacion'></a>
## Preparación de datos

In [1]:
import pandas as pd
import numpy as np

from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

from IPython.display import display

Los datos de exploración geológica de las tres regiones se almacenan en archivos:

In [2]:
# Carga de datos
try:
    zone_a = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_0.csv')
    zone_b = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_1.csv')
    zone_c = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_2.csv')

except:
    zone_a = pd.read_csv('geo_data_0.csv')
    zone_b = pd.read_csv('geo_data_1.csv')
    zone_c = pd.read_csv('geo_data_2.csv')

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Excelente importación de las librerías, y a la vez muy bien importados los datos manteniéndolos separados de las importaciones!

**Características de los dataframes:**

* `id` — identificador' único de pozo de petróleo
* `f0, f1, f2` — tres características de los puntos
* `product` — volumen de reservas en el pozo de petróleo (miles de barriles).

In [4]:
# Muestra de datos

def preview(df):
    df.info()
    print('\nNúmero de filas duplicadas:', df.duplicated().sum())
    display(df.describe(percentiles=[]).round(3))
    display(df.head())

In [5]:
preview(zone_a)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Número de filas duplicadas: 0


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.500,0.250,2.503,92.500
std,0.872,0.504,3.248,44.289
min,-1.409,-0.848,-12.088,0.000
50%,0.502,0.250,2.516,91.850
max,2.362,1.344,16.004,185.364


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


El df de la **Zona A** cuenta con 100,000 observaciones. No se encontraron datos ausentes ni observaciones repetidas.

In [6]:
preview(zone_b)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Número de filas duplicadas: 0


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,1.141,-4.797,2.495,68.825
std,8.966,5.120,1.704,45.944
min,-31.610,-26.359,-0.018,0.000
50%,1.153,-4.813,2.011,57.086
max,29.422,18.734,5.020,137.945


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


El df de la **Zona B** cuenta con 100,000 observaciones. No se encontraron datos ausentes ni observaciones repetidas.

In [7]:
preview(zone_c)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Número de filas duplicadas: 0


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.002,-0.002,2.495,95.000
std,1.732,1.730,3.473,44.750
min,-8.760,-7.084,-11.970,0.000
50%,0.009,-0.009,2.484,94.926
max,7.238,7.845,16.739,190.030


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


El df de la **Zona C** cuenta con 100,000 observaciones. No se encontraron datos ausentes ni observaciones repetidas.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Excelente implementación de los métodos para observar la composición de los datos! A la vez excelente corroboración de duplicados.

<a id='aplicacion'></a>
## Aplicación de modelo de regresión lineal por zona

En esta sección, se ha diseñado un modelo de regresión lineal que utilizará el 75% de los datos proporcionados de cada zona como datos de entrenamiento, reservando el 25% restante como datos de validación. Posteriormente, se presentarán el volumen medio de reservas predicho y el RECM del modelo.

In [8]:
def model(df, pred):
    features = df.drop(['id','product'], axis=1)
    target = df['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    print ("Volumen medio de reservas real:    ", target_valid.mean().round(2))
    print ("Volumen medio de reservas predicho:", predictions_valid.mean().round(2))
    print ("RECM del modelo de regresión lineal en el conjunto de validación:", result.round(2))
    return pred.append(predictions_valid)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Aquí varias cosas bien hechas Alejandro, por un lado excelente al no tomar en cuenta la columna id que no nos aporta valor al modelo, por otro lado una excelente divisón de datos con su posterior entrenamiento y calculo de metrica, excelente!

In [9]:
pred_a = []
model(zone_a, pred_a)

Volumen medio de reservas real:     92.08
Volumen medio de reservas predicho: 92.59
RECM del modelo de regresión lineal en el conjunto de validación: 37.58


Estos resultados indican que el modelo de regresión lineal ha logrado predecir con precisión el volumen medio de reservas, con una diferencia de solo 0.51 unidades de barriles entre el valor real y el predicho. Esto sugiere que el modelo ha capturado adecuadamente la relación entre las variables predictoras y la variable de respuesta. Por otro lado, **la RECM de 37.58 unidades** de barriles en el conjunto de validación muestra que, en promedio, las predicciones del modelo difieren en aproximadamente esa cantidad del valor real correspondiente. **Se trata de un error relativamente alto, teniendo en cuenta que el volumen medio real de reservas es de alrededor de 92**.

In [10]:
pred_b = []
model(zone_b, pred_b)

Volumen medio de reservas real:     68.72
Volumen medio de reservas predicho: 68.73
RECM del modelo de regresión lineal en el conjunto de validación: 0.89


El modelo de regresión lineal diseñado para predecir el volumen de reservas de pozos petroleros ha arrojado resultados interesantes. En primer lugar, tanto el volumen medio de reservas reales como el predicho son muy similares, con un valor real de 68.72 y un valor predicho de 68.73 unidades. Esto sugiere que el modelo ha logrado una buena precisión en la predicción del volumen de reservas. **La RECM del modelo es de 0.89, lo que indica que el modelo es capaz de realizar predicciones precisas**, ajustándose bien a los datos y siendo efectivo en la predicción del volumen de producción de pozos petroleros.

In [11]:
pred_c = []
model(zone_c, pred_c)

Volumen medio de reservas real:     94.88
Volumen medio de reservas predicho: 94.97
RECM del modelo de regresión lineal en el conjunto de validación: 40.03


El volumen medio de reservas reales es de 94.88, mientras que el volumen medio de reservas predicho es de 94.97. La ligera disparidad entre los valores reales y predichos sugiere que el modelo de regresión lineal ha demostrado una sólida capacidad predictiva, dado que la discrepancia entre los valores es mínima. Esto indica que el modelo ha logrado capturar de manera efectiva las tendencias y patrones presentes en los datos. No obstante, **la RECM del modelo es de 40.03, una cifra considerablemente alta en comparación con el volumen medio real de producción de 94.88 unidades**.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Llegamos a los valores esprados, bien hecho!

<a id='puntos'></a>
## Cálculo de puntos potenciales por zona

Dado que la inversión total es de 100 millones de dólares para desarrollar 200 pozos petrolíferos, cada pozo requiere una inversión promedio de 500,000 dólares. Para evitar pérdidas, cada pozo debe producir al menos un valor de 500,000 dólares (equivalente a 111.1 unidades de 1,000 barriles cada una) al precio de venta de $4,500 por unidad.

* Presupuesto total: \$100,000,000 USD
* Total de pozos a desarrollar: 200 pozos
* Presupuesto por pozo: \$500,000 USD
* Precio venta x unidad de barriles (1,000 barilles): \$4,500 USD
* Punto de equilibrio: 111.1 unidades

In [12]:
investment = 1e8
wells = 200
mean_investment = investment/wells
unit_value = 4500
balance = mean_investment/unit_value

A continuación, se ha diseñado una función para calcular la media y la mediana de la producción de petróleo en cada zona, y determinar el cuantil que cumple con la producción mínima de 111.1 unidades.

In [13]:
def units(zone):
    product = zone['product']
    quantile_number = (product[product <= balance].count() / len(product))
    print('Media de producción en la zona:', product.mean().round(2))
    print('Mediana de producción en la zona:', product.median().round(2))
    print('Cuantil de equilibrio (111.1 unidades):', quantile_number.round(3)*100)

In [14]:
units(zone_a)

Media de producción en la zona: 92.5
Mediana de producción en la zona: 91.85
Cuantil de equilibrio (111.1 unidades): 63.4


La media de producción de petróleo por punto en la **zona petrolífera A** es de 92.5 unidades, mientras que la mediana de producción es de 91.85 unidades. La proximidad entre ambas cifras sugiere una distribución bastante simétrica de los datos de producción. A pesar de esto, ambas cifras se sitúan por debajo del mínimo requerido de 111.1 unidades, que es el punto de equilibrio para evitar pérdidas. No obstante, el valor del cuantil 63.4 es de 111.1 unidades, lo que indica que, aunque la mayoría de los puntos producen menos de 111.1 unidades, **aproximadamente 36,600 puntos cumplen o superan el requisito mínimo de 111.1 unidades**.

In [15]:
units(zone_b)

Media de producción en la zona: 68.83
Mediana de producción en la zona: 57.09
Cuantil de equilibrio (111.1 unidades): 83.5


La producción media de petróleo por punto en la **zona petrolífera B** es de 68.83 unidades, mientras que la mediana de producción es de 57.09 unidades. Esta disparidad entre la media y la mediana indica un sesgo positivo en los datos, lo que sugiere que la presencia de valores atípicos está influyendo en la media y la hace no representativa de la tendencia central de los datos. A pesar de que ambas cifras se sitúan por debajo del mínimo requerido de 111.1 unidades para alcanzar el punto de equilibrio y evitar pérdidas, el cuantil 83.5 muestra un valor de 111.1 unidades. Esto implica que, aunque la mayoría de los puntos producen menos de 111.1 unidades, **aproximadamente 16,500 puntos cumplen o superan el requisito mínimo establecido**.

In [16]:
units(zone_c)

Media de producción en la zona: 95.0
Mediana de producción en la zona: 94.93
Cuantil de equilibrio (111.1 unidades): 61.8


La producción media de petróleo por pozo en la **zona petrolífera C** es de 95.0 unidades, mientras que la mediana de producción es de 94.93 unidades. La proximidad entre ambas cifras sugiere una distribución bastante simétrica de los datos de producción. A pesar de ello, tanto la media como la mediana se sitúan por debajo del mínimo requerido de 111.1 unidades, que es el punto de equilibrio para evitar pérdidas. No obstante, el valor del percentil 61.8 es de 111.1 unidades, lo que indica que **aproximadamente el 38,200 de los pozos cumplen o superan el requisito mínimo. Esto convierte a esta zona en la que tiene más pozos petrolíferos que producen las unidades suficientes para alcanzar al menos el punto de equilibrio**.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Excelente forma de corroborar los puntos de equilibrio, bien hecho Alejandro!

<a id='ganancias'></a>
## Cálculo de ganancias estimadas

A continuación, se seleccionarán los 200 pozos con los valores de predicción más altos, basándose en los datos originales de exploración geológica de cada zona. A partir de esta información, se calculará el promedio de producción de unidades proyectadas por punto y las ganancias:

* Ingresos totales promedio = promedio de unidades x precio de venta (\$4,500 USD)
* Beneficio bruto promedio = ingresos totales - inversión inicial (\$500,000 USD)
* Margen de beneficio bruto promedio

Estos cálculos nos permitirán identificar los pozos más rentables y tomar decisiones informadas para maximizar la eficiencia y rentabilidad en la explotación de recursos.

In [17]:
def revenue(zone, best_200):
    
### CAMBIOS ####

    # 1. Predicciones para TODO el df

    features = zone.drop(['id','product'], axis=1)
    target = zone['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    zone['prediction'] = model.predict(features)
    
    # 2. Ordenar y seleccionar top 200
        
    #no# df = pd.Series(zone['product'])
    #no# result = df.explode().reset_index(drop=True).sort_values(ascending=False)[:200]
    
    zone_pred_200 = zone.sort_values(by='prediction', ascending=False)[:200]
    twohundred_product = zone_pred_200['product'].sum().round(2)
   
    # 3. Totales en vez de promedios
    total_income = (twohundred_product * unit_value).round(2)
    net_profit = total_income - investment
    margin_income = (net_profit/total_income)*100

    print(f'''
Inversión inicial: ${investment:,} USD

Unidades producidas en los mejores {wells} puntos: {twohundred_product:,} unidades

Ganancias proyectadas: 
* Ingresos totales:          ${total_income:,} USD
* Beneficio bruto:           ${net_profit:,} USD
* Margen de beneficio bruto: {margin_income.round(2)}%
''')
    
    return best_200.append(zone_pred_200)

**Zona A**

In [18]:
zone_a_pred_200 = []
revenue(zone_a, zone_a_pred_200)  


Inversión inicial: $100,000,000.0 USD

Unidades producidas en los mejores 200 puntos: 29,986.9 unidades

Ganancias proyectadas: 
* Ingresos totales:          $134,941,050.0 USD
* Beneficio bruto:           $34,941,050.0 USD
* Margen de beneficio bruto: 25.89%



**Zona B**

In [19]:
zone_b_pred_200 = []
revenue(zone_b, zone_b_pred_200)


Inversión inicial: $100,000,000.0 USD

Unidades producidas en los mejores 200 puntos: 27,589.08 unidades

Ganancias proyectadas: 
* Ingresos totales:          $124,150,860.0 USD
* Beneficio bruto:           $24,150,860.0 USD
* Margen de beneficio bruto: 19.45%



**Zona C**

In [20]:
zone_c_pred_200 = []
revenue(zone_c, zone_c_pred_200)


Inversión inicial: $100,000,000.0 USD

Unidades producidas en los mejores 200 puntos: 27,936.47 unidades

Ganancias proyectadas: 
* Ingresos totales:          $125,714,115.0 USD
* Beneficio bruto:           $25,714,115.0 USD
* Margen de beneficio bruto: 20.45%



**Conclusiones**

Basándonos en los datos obtenidos a partir de los 200 mejores puntos de cada zona, la **Zona A parece ser la mejor opción de inversión debido a su alto promedio de unidades proyectadas, ingresos totales y beneficio bruto, junto con un margen de beneficio bruto estimado del 31.93%**. Esta zona ofrece un equilibrio entre producción y rentabilidad, lo que la convierte en una opción atractiva para maximizar las ganancias.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-warning">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Has hecho un gran procedimiento aqui Alejandro, sin embargo debemos corregir algunos detalles.
    
1. Por un lado, debemos tener en cuenta que si debemos tomar los top 200 pozos y sus valores reales tal como lo hiciste pero para elegir estos pozos no debemos hacerlo por su valor real sino la predicción que estamos realizando. Te recomiendo **agregar a tu dataframe las predicciones de los pozos, ordenar por esta feature y ahí agarrar los top 200 valores reales**.
    
2. En segundo lugar tengamos en cuenta que al estar calculando ganancias de regiones debemos **al total_income restarle su presupuesto**, nosotros estamos restando el promedio por pozo (mean_investment) pero eso representa un pozo unicamente pero son **200** en realidad.

<div class="alert alert-block alert-warning">
<b>Comentario del revisor (Iteración 2).</b> <a class="tocSkip"></a>

Un gran avance Alejandro ya que has resuelto el punto 1 sin embargo seguimos aún calculando un promedio en la ganancia, del resultado de zone_pred_200 no debemos tomar **predictions** sino que su valor real **prudct**, luego sobre estos top 200 valores debemos aplicarle el método **sum()**, a esto si multiplicarlo por los 4500 y luego restarle los 100.000.000.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (Iteración 2).</b> <a class="tocSkip"></a>

Felicitaciones Alejandro! Ahora si, muy bien hecho!

<div class="alert alert-block alert-info">
<b>Respuesta</b> <a class="tocSkip"></a>

Ahora sí! También ya veo la lógica detrás de todo esto.

</div>

<a id='intervalos'></a>
## Cálculo de intervalos de confianza y riesgos para cada región

<div class="alert alert-block alert-info">
<b>Comentario</b> <a class="tocSkip"></a>
    
Batallé mucho para hacer esta sección. Me parece un poco extraño verificar si la probabilidad es menor al 2.5%, ya que si consideramos que ya hemos seleccionado los mejores puntos, entonces la probabilidad de tener pérdidas debería ser nula, especialmente si nuestro objetivo es alcanzar las 111.1 unidades por punto. Sería muy distinto si se seleccionaran muestras de los dfs originales, pero entonces, ¿qué caso tendría haber seleccionado previamente los mejores? 
    
De todas formas, aquí está mi mejor intento. Por favor, si encuentras algún error, agradecería observaciones detalladas, ya que me estoy reintegrando a este curso después de muchos meses y me siento un poco perdido en general. Gracias mil.
</div>

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Tengamos en cuenta Alejandro que para esta sección no estamos tomando las top 200 sino que estamos tomando 500 muestras aleatorias en base a sus valores reales, de esta forma podemos asegurar la consistencia de la región.

En la siguiente sección, se diseñó una función para aplicar la técnica del bootstrapping con 1,000 muestras de 500 puntos y analizar la distribución de los beneficios en ellas.  Esta función proporcionará el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas para cada zona, ofreciendo una visión detallada de las posibles ganancias y pérdidas en cada área.

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

A no preocuparnos Alejandro, vienes genial a pesar de haber pasado tiempo, se nota un gran manejo de las herramientas. Primero comenzando por tu duda, tiene sentido lo que dices y esto se deriva porque estamos pensando en pasar nuestros 200 pozos al calculo de riesgo, sin embargo el objetivo es pasar la región entera para que sobre esta seleccionemos muestras aleatorias y sobre ellas hacer estos calculos, al ser muestras aleatorias sobre el total podemos luego ver un resultado más seguro al respecto.

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Ahora yendo al calculo Alejandro, en general esta muy bien aunque debemos corregir algunas cosas, vayamos paso a paso:
    
1. Debemos tomar **toda la región** y no unicamente los 200 pozos.
2. Las 1000 iteraciones son correctas, sin embargo el hecho de implementar frac=1 significa que estamos tomando el 100% de los datos, esto haría que todas las muestras sean siempre igual, por esta razón deberíamos **aplicar n=500** que tomara aleatoriamente 500 pozos.
3. Los valores que debemos calcular sobre la muestra aleatoria no es el promedio sino que deberiamos **calcular el beneficio de dichos pozos utilizando nuestra función anteriormente creada**.
4. Luego recordemos calcular la perdida, que podríamos hacerlo de la siguiente forma por ejemplo **perdida=((pd.Series(values)<0).mean())**
5. Por ultimo los calculos de los intervalores son correctos, si deseas puedes guiarte por una forma más sencilla como la siguiente: 
                                                                                                                                            alpha = 0.05 
                                                                                                                                            interv_low = serie_benef.quantile(alpha/2) 
                                                                                                                                            interv_up = serie_benef.quantile(1-alpha/2)  
                                                                                                                                      

<div class="alert alert-block alert-info">
<b>Comentario</b> <a class="tocSkip"></a>
    
Muchas gracias por tus instrucciones. Espero haberlas aplicado correctamente. Por lo menos siento que veo una luz al final del túnel... espero que no sea un tren.
    
</div>

In [22]:
def income_risk(zone):
    
    data = pd.Series(zone['prediction']).reset_index(drop=True)
    state = np.random.RandomState(12345)
    
    ### De 500 muestras aleatorias SIN escoger las 200 mejores
    # Bootstraping
    values = []
    for i in range(1000):
        bootstrap = data.sample(n=500, replace=True, 
                                random_state=state).sort_values(ascending=False)[:200]
        revenue = (bootstrap * unit_value).sum()
        values.append((revenue - investment).round(2))
    
    # Intervalo de confianza
    benefits = pd.Series(values)
    
    alpha = 0.05 
    interv_low = benefits.quantile(alpha/2).round(2)
    interv_up = benefits.quantile(1-alpha/2).round(2)
    
    # Riesgo de pérdidas
    
    #loss_probability = sum(value < (111.1*4500) for value in values) / len(values)
    loss=(((pd.Series(values) < 0).mean())* 100).round(2)

        
    # Muestra de resultados
    print(f'''
Beneficio medio proyectado: ${(benefits.mean().round(2)):,} USD
Intervalo de confianza del 95%: (${interv_low:,} USD, ${interv_up:,} USD)
Riesgo de pérdidas: {loss:,}%''')

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Excelente implementación de lo solicitado Alejandro, muy bien hecho!

**Zona A**

In [23]:
income_risk(zone_a)


Beneficio medio proyectado: $3,586,406.81 USD
Intervalo de confianza del 95%: ($1,294,396.34 USD, $5,825,535.73 USD)
Riesgo de pérdidas: 0.0%


**Zona B**

In [24]:
income_risk(zone_b)


Beneficio medio proyectado: $4,469,223.75 USD
Intervalo de confianza del 95%: ($395,594.56 USD, $8,267,349.83 USD)
Riesgo de pérdidas: 1.4%


**Zona C**

In [25]:
income_risk(zone_c)


Beneficio medio proyectado: $2,807,007.65 USD
Intervalo de confianza del 95%: ($887,613.53 USD, $4,872,695.5 USD)
Riesgo de pérdidas: 0.1%


**Conclusiones**

Al utilizar las 200 mejores proyecciones de una muestra aleatoria de 500 puntos, las zonas A y B se perfilan como buenas zonas de inversión. Por un lado, la Zona A tiene un beneficio medio proyectado de \\$3,586,406.81 USD con un riesgo practicamente nulo. Por su parte, **los beneficios esperados de la zona B son mayores, ya que rondan los  \\$4.469.223,75 USD (+$882,816.94). Aunque la Zona B conlleva un riesgo de pérdida del 1,4%, es relativamente bajo y puede ser una compensación aceptable por los beneficios potenciales significativamente mayores**. No obstante, es importante tener en cuenta otros factores más allá de las cifras de beneficios y riesgo, como el horizonte temporal, las necesidades de liquidez y las estrategias de inversión global.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Lo has hecho muy bien Alejandro, has llegado a la conclusión que se buscaba, la implementación fue correcta, si bien podríamos modificar algunos pasos para que sea más ordenado lo has implementado correctamente.

<a id='conclusiones'></a>
## Conclusiones generales

* El presente estudio tuvo como objetivo identificar los mejores lugares para la apertura de 200 nuevos pozos petroleros dentro de tres zonas geológicas.


* Los conjuntos de datos proporcionados por la empresa constaban de 100,000 observaciones cada uno, sin datos faltantes ni observaciones repetidas.


* Se entrenó y aplicó un modelo de regresión para cada una de las tres zonas. En el caso de la zona B, el modelo tuvo una RECM de 0.89, lo que indica una alta precisión en las predicciones. Sin embargo, para las otras dos zonas, el RECM fue aproximadamente de 40 unidades, sugiriendo margen de mejora en el modelo.


* Posteriormente, se calcularon la media y la mediana de producción de cada zona, así como el cuantil que marca el punto de equilibrio. Las zonas A y C mostraron más de 36,000 puntos potenciales donde se proyecta alcanzar al menos el punto de equilibrio.


* A continuación, se seleccionarán los 200 pozos con los valores de predicción más altos, basándose en los datos originales de exploración geológica de cada zona. A partir de esta información, se calculó el promedio de producción de unidades proyectadas por punto, así como sus correspondientes beneficios. **La zona A resultó la más rentable, con un promedio de producción proyectada de 163.22 unidades por punto, lo que supondría un beneficio bruto de $46,898,000.0 USD (margen bruto del 31.93%).**


* Finalmente, se calcularon el intervalo de confianza y los riesgos para cada región a partir de tomar los 200 mejores puntos de una muestra aleatoria de 500 puntos. Esta vez, **la zona B se destacó como la mejor opción, ya que el intervalo de confianza del 95% mostró un intervalo que va de los \\$395,594.56 USD a los \\$8,267,349.83 USD), y un riesgo relativamente bajo de 1.4%**.

[Volver al inicio](#contenido)

<div class="alert alert-block alert-success">
<b>Comentario del revisor.</b> <a class="tocSkip"></a>

Quería dedicarme aquí a agradecerte Alejandro por que has hecho un trabajo de exelencia, desde las implementaciones detalladas y los agregados a lo solicitado en el proyecto. A la vez destacar tanto tus conclusiones parciales como esta final que han complentado y demostrado tu gran comprensión de lo trabajado y lo obtenido.    

<div class="alert alert-block alert-info">
<b>Comentario</b> <a class="tocSkip"></a>
    
Muchas gracias por tus comentarios, me han dado muchos ánimos, sobre todo cuando me sentía confundido. Es cierto que conforme iba haciendo el proyecto me sentía más seguro y recordaba cosas. Vamos a ver qué tal va con esta iteración.
    
</div>